In [1]:
import sys
sys.path.append("..")
from pathlib import Path
from utils.visualise import get_ocr_stat, display_ocr, display_folder
from utils.config import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Iterator

In [2]:
path_data_train = "../data/train"
path_data_val = "../data/val"
path_data_test = "../data/test"
# path_data_test = '../data/aldo/test/'
 
path_folders_train = "../data/train_folders.txt"
path_folders_val = "../data/val_folders.txt"
path_folders_test = "../data/test_folders.txt"
# path_folders_test = '../data/aldo_test_folders.txt'

In [3]:
folders_test = np.array([line.split() for line in open(path_folders_test).readlines(  )])
folders_test

array([['TDJC01014', '1'],
       ['TDJC00915', '1'],
       ['TDJC00655', '1'],
       ['TDJC01079', '1'],
       ['TDJC00659', '1'],
       ['TDJC00909', '1'],
       ['TDJC00793', '1'],
       ['TDJC01361', '1'],
       ['TDJC01679', '1'],
       ['TDJC00793', '2'],
       ['TDJC00424', '2'],
       ['TDJC01768', '2'],
       ['TDJC01725', '2'],
       ['TDJC00787', '2'],
       ['TDJC00612', '2'],
       ['TDJC00499', '2'],
       ['TDJC01281', '2'],
       ['TDJC00368', '2'],
       ['TDJC00915', '2'],
       ['TDJC01725', '2'],
       ['TDJC00719', '2'],
       ['TDJC00612', '2'],
       ['TDJC01596', '3'],
       ['TDJC00830', '3'],
       ['TDJC01195', '3'],
       ['TDJC00659', '3'],
       ['TDJC01659', '3'],
       ['TDJC00719', '3'],
       ['TDJC00613', '3'],
       ['TDJC00468', '3'],
       ['TDJC00719', '3'],
       ['TDJC00754', '3'],
       ['TDJC00468', '3'],
       ['TDJC00203', '3'],
       ['TDJC01659', '3'],
       ['TDJC00275', '3'],
       ['TDJC01390', '3'],
 

In [4]:

def make_labels_array(glob: Iterator[Path]) -> np.array:
    out_arr = []
    for path in glob:
        split = path.stem.split('-')
        if len(split) == 2:
            split.append('multi')
        else:
            split.extend(['0','single'])
        out_arr.append(split)
        
    return np.array(out_arr)
        

In [5]:
pages_test_img = make_labels_array(Path(path_data_test).rglob("*jpg"))
pages_test_ocr = make_labels_array(Path(path_data_test).rglob("*tsv"))
assert (
    pages_test_ocr.shape == pages_test_img.shape
), f"number of pages in ocr and images not match"

folders_test = np.array([line.split() for line in open(path_folders_test).readlines()])
assert (
    np.unique(folders_test[:, 0]).shape[0] == np.unique(pages_test_ocr[:, 0]).shape[0]
), f"number of documents in files and txt not match"

print("Test set:")
print(
    f"   found {np.unique(folders_test[:,0]).shape[0]} documents, {pages_test_ocr.shape[0]} pages, {np.unique(folders_test[:,1]).shape[0]} folders"
)

Test set:
   found 89 documents, 195 pages, 20 folders


In [6]:
# pages_test_ocr = np.array(
#     [
#         (path.stem.split("-") if len(path.stem.split("-")) == 2 else [path.stem, "0"])
#         for path in Path(path_data_test).glob("**/*.tsv")
#     ]
# )
# pages_test_img = np.array(
#     [
#         (
#             path.stem.split("-").extend("multi")
#             if len(path.stem.split("-")) == 2
#             else [path.stem, "0", "single"]
#         )
#         for path in Path(path_data_test).glob("**/*.jpg")
#     ]
# )
# assert (
#     pages_test_ocr.shape == pages_test_img.shape
# ), f"number of pages in ocr and images not match"

# folders_test = np.array([line.split() for line in open(path_folders_test).readlines()])
# assert (
#     np.unique(folders_test[:, 0]).shape[0] == np.unique(pages_test_ocr[:, 0]).shape[0]
# ), f"number of documents in files and txt not match"

# print("Test set:")
# print(
#     f"   found {np.unique(folders_test[:,0]).shape[0]} documents, {pages_test_ocr.shape[0]} pages, {np.unique(folders_test[:,1]).shape[0]} folders"
# )

In [7]:
np.unique(folders_test[:,0]).shape[0], np.unique(pages_test_ocr[:,0]).shape[0]

(89, 89)

In [8]:
df = pd.DataFrame({"id": pages_test_img[:,0], "page_num": pages_test_img[:,1], 'doc_type': pages_test_img[:, 2]})

df['stem'] = np.where(df['doc_type'] == 'single', df['id'] , df['id'] + '-' + df['page_num'])
# df["stem"] = df.id + '-' + df.page_num
df["label"] = (df["page_num"].astype(int)==0).astype(int)
df = df.drop('doc_type', axis=1)

In [9]:
df1 = pd.DataFrame({"id": folders_test[:,0], "folder_id": folders_test[:,1]})

In [10]:
df1 = df1.merge(df, on = "id", how = "left")

In [11]:
folder = df1[["folder_id", "id"]].groupby("folder_id").count()
folder.rename(columns = {"id":"merged_num_of_pages"}, inplace = True)
df1 = df1.merge(folder, on = "folder_id")

In [12]:
df1.head(50)

,id,folder_id,page_num,stem,label,merged_num_of_pages
0,TDJC01014,1,1,TDJC01014-1,0,25
1,TDJC01014,1,0,TDJC01014-0,1,25
2,TDJC00915,1,0,TDJC00915-0,1,25
3,TDJC00915,1,1,TDJC00915-1,0,25
4,TDJC00655,1,1,TDJC00655-1,0,25
5,TDJC00655,1,0,TDJC00655-0,1,25
6,TDJC00655,1,2,TDJC00655-2,0,25
7,TDJC01079,1,0,TDJC01079-0,1,25
8,TDJC01079,1,1,TDJC01079-1,0,25
9,TDJC00659,1,4,TDJC00659-4,0,25


In [13]:
folder.mean()

merged_num_of_pages    27.6
dtype: float64

In [14]:
df1.to_csv("full_model_prediction.csv")